In [150]:
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from datetime import datetime, timedelta

In [204]:
SCOPES = [
    'https://www.googleapis.com/auth/calendar.events',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/calendar.readonly'
]

def main():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=261808777338-vqdke41pbqeccdbd20v1tdeojnaqgu85.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.events+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=WiHYD2lhRxgG7hcdQ6mwDv5OblY97O&access_type=offline


In [205]:
def get_document_content(document_id):
    creds = Credentials.from_authorized_user_file('token.json')

    # SCOPES remains the same
    service = build('docs', 'v1', credentials=creds)

    try:
        document = service.documents().get(documentId=document_id).execute()
        content = document.get('body').get('content')
        return content
    except Exception as e:
        print(f"Error retrieving document content: {e}")
        return None

# Example usage:
document_id = 'YOUR_DOCUMENT_ID'  # Replace with your actual document ID
content = get_document_content('1Yaq9bvvwEVlF88rKYFdksqf9Hf6OQjJTEgeXnvFFdGQ')

if content:
    print("Document Content:")
    print(content)
    output = content[1]['paragraph']['elements'][0]['textRun']['content']
else:
    print("Failed to retrieve document content.")

Error retrieving document content: <HttpError 403 when requesting https://docs.googleapis.com/v1/documents/1Yaq9bvvwEVlF88rKYFdksqf9Hf6OQjJTEgeXnvFFdGQ?alt=json returned "The caller does not have permission". Details: "The caller does not have permission">
Failed to retrieve document content.


In [215]:
output

In [216]:
import openai
api_key = "sk-proj-FoSaiCmz1LDP6hk6KNYBT3BlbkFJw09Im1RN56SUbv31peBR"

from datetime import datetime
today = str(datetime.utcnow())

In [217]:
from openai import OpenAI
import json

client = OpenAI(api_key = api_key)
response = ""
example = {
  "appointment_information": "Appointment with John Doe",
  "start_date_time": "2024-05-04T08:00:00",
  "end_date_ime": "2024-05-04T09:00:00"
}
example = json.dumps(example)

stream = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", 
         "content": "you only give in a json format with appointment_information, start_date_time(iso_format) and end_date_time(iso_format), today's date is"+today+ "example: "+ example +output +"\n now read the above text and give me the response, response MUST be formatted in JSON"
        }
         ],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
        response += chunk.choices[0].delta.content

{
"appointment_information": "Appointment with Hemanth",
"start_date_time": "2024-05-05T22:55:00",
"end_date_time": "2024-05-05T23:55:00"
}

In [218]:
print(response)

{
"appointment_information": "Appointment with Hemanth",
"start_date_time": "2024-05-05T22:55:00",
"end_date_time": "2024-05-05T23:55:00"
}


In [219]:
import json
details = json.loads(response)
print(details)

{'appointment_information': 'Appointment with Hemanth', 'start_date_time': '2024-05-05T22:55:00', 'end_date_time': '2024-05-05T23:55:00'}


In [220]:
# Load credentials from token.json
creds = Credentials.from_authorized_user_file('token.json')

# Build the service
service = build('calendar', 'v3', credentials=creds)

# Retrieve a list of calendars
calendar_list = service.calendarList().list().execute()

# Iterate through each calendar and print its timezone
for calendar in calendar_list.get('items', []):
    print('Calendar ID:', calendar['id'])
    print('Timezone:', calendar.get('timeZone', 'Unknown'))

Calendar ID: poornavenkat.n@gmail.com
Timezone: Asia/Kolkata
Calendar ID: addressbook#contacts@group.v.calendar.google.com
Timezone: Asia/Kolkata
Calendar ID: en-gb.indian#holiday@group.v.calendar.google.com
Timezone: Asia/Kolkata


In [221]:
time_zone = 'UTC'

In [222]:
# Load credentials from token.json
creds = Credentials.from_authorized_user_file('token.json')
calendar = build('calendar', 'v3', credentials=creds)

# Define appointment details
appointment_summary = details['appointment_information']
# appointment_start = datetime.strptime(details['start_date'], "%Y-%m-%dT%H:%M:%S")
# appointment_end = datetime.strptime(details['end_date'], "%Y-%m-%dT%H:%M:%S")

# Replace 'Your_Time_Zone' with the actual timezone, e.g., 'America/New_York'

# Create event body
event = {
    'summary': appointment_summary,
    'start': {'dateTime': details["start_date_time"], 'timeZone': time_zone},
    'end': {'dateTime': details["end_date_time"], 'timeZone': time_zone}
}

# Insert event
event = calendar.events().insert(calendarId='primary', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))


Event created: https://www.google.com/calendar/event?eid=ZmJyN3BxNzhpNDY1NWg4YzlvcTg2Y2liMTAgcG9vcm5hdmVua2F0Lm5AbQ


In [223]:
details

{'appointment_information': 'Appointment with Hemanth',
 'start_date_time': '2024-05-05T22:55:00',
 'end_date_time': '2024-05-05T23:55:00'}